In [2]:
from binance import Client

import numpy as np
import pandas as pd
import ta 
import time

In [3]:
api_key = 's20c6FhOSUORTSgkjROWRCekmWoyMtMnV3BEbistCKvX3OUckd4vkL1WCNF6iZ5x'
secret_key = 'vBmwXWkWTM5hfLbury8sMAfgI6Zfnw3ZLkq524Cve0KHwUpWNdeYcz21VdzTW69E'

client = Client(api_key, secret_key)

In [4]:
stream = pd.DataFrame(columns=['Name', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [5]:
pairs = []
exchange_info = client.get_exchange_info()
for s in exchange_info['symbols']:
     if (s['symbol'][-4:] == 'USDT') & (s['status'] == 'TRADING') & ((s['symbol'][-6:-4] != 'UP') & (s['symbol'][-8:-4] != 'DOWN')):
        pairs.append(s['symbol'])

In [6]:
def get_data(symbol, interval, lookback):
    df = pd.DataFrame(client.get_historical_klines(symbol,
                                      interval,
                                      str(lookback) + ' minute ago UTC'))
    df = df.iloc[:,:6]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.insert(loc=0, column='Name', value=symbol)
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    df.Time = pd.to_datetime(df.Time, unit = 'ms')
    # df.set_index('Time', inplace = True)
    
    return df

In [7]:
def get_ta(df):
    df['ema'] = ta.trend.ema_indicator(df.Close, window = 200)
    df['macd_line'] = ta.trend.macd(df.Close)
    df['macd_signal'] = ta.trend.macd_signal(df.Close)
    df['macd_diff'] = ta.trend.macd_diff(df.Close)
    df['prev_macd'] = df['macd_diff'].shift(1)
    return df

In [7]:
# for pair in pairs:
pair_data = get_ta(get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '10000'))


In [8]:
signal = []
for ind, row in pair_data.iterrows():
    buy_cond_1 = (row.iloc[10] > 0) & (row.iloc[11] < 0) 
    buy_cond_2 = (row.iloc[8] < 0) & (row.iloc[9] < 0)
    # buy_cond_3 = (row['Close'] > row['ema'])

    sell_cond_1 = (row.iloc[10] < 0) & (row.iloc[11] > 0)
    sell_cond_2 = (row.iloc[8] > 0) & (row.iloc[9] > 0)
    # sell_cond_3 = (row['Close'] < row['ema'])

    if buy_cond_1 & buy_cond_2:# & buy_cond_3:
        signal.append(1)
    elif sell_cond_1 & sell_cond_2:# & sell_cond_3:
        signal.append(-1)
    else:
        signal.append(0)
signal

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [9]:
signal = pd.Series(signal, name="buy/sell")
pair_data = pd.concat([pair_data,signal], axis = 1)

In [10]:
triggered = pair_data[(pair_data["buy/sell"] == 1) | (pair_data["buy/sell"] == -1)]

In [11]:
triggered

,Name,Time,Open,High,Low,Close,Volume,ema,macd_line,macd_signal,macd_diff,prev_macd,buy/sell
45,BTCUSDT,2022-12-23 12:40:00,16861.20,16862.24,16860.01,16861.61,44.81356,NaN,3.848745,3.886549,-0.037803,0.128818,-1
70,BTCUSDT,2022-12-23 13:05:00,16874.24,16875.48,16872.42,16874.01,53.11878,NaN,4.023935,4.049762,-0.025828,0.119209,-1
86,BTCUSDT,2022-12-23 13:21:00,16867.91,16871.14,16866.39,16866.86,63.96291,NaN,1.078733,1.311488,-0.232755,0.082536,-1
93,BTCUSDT,2022-12-23 13:28:00,16870.65,16871.56,16865.00,16865.69,97.16706,NaN,0.790938,1.109122,-0.318184,0.122520,-1
104,BTCUSDT,2022-12-23 13:39:00,16849.39,16907.75,16847.24,16887.20,1366.18060,NaN,-7.969954,-8.751434,0.781480,-4.044209,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9904,BTCUSDT,2022-12-30 08:59:00,16501.34,16501.88,16497.08,16500.24,83.15779,16517.117486,5.685369,5.860889,-0.175520,0.245605,-1
9932,BTCUSDT,2022-12-30 09:27:00,16519.35,16521.54,16518.77,16519.68,41.83921,16516.145490,5.479078,5.652394,-0.173316,0.215060,-1
9960,BTCUSDT,2022-12-30 09:55:00,16519.88,16521.37,16518.85,16520.27,40.65972,16516.745265,-0.043363,-0.210767,0.167404,-0.110659,1
9975,BTCUSDT,2022-12-30 10:10:00,16520.70,16521.88,16515.99,16516.48,101.65052,16517.178457,0.874339,0.976156,-0.101817,0.312991,-1


In [211]:
# correct = 0
# wrong = 0
# sale_cost = 0
# amount = 0

# for i in range(len(stream)):
#     if(stream.iloc[i,12] == 1):
#         amount += 1
#         sale_cost += stream.iloc[i,5]
#         print("Bought {} at an average of {}".format(amount, sale_cost/amount))
#     elif((stream.iloc[i,12] == -1) | (stream.iloc[i,5] <= sale_cost *0.98)):
#         if amount != 0 :
#             sale_rev = amount * stream.iloc[i,5]
#             print("Sold {} at {}".format(amount, stream.iloc[i,5]))
#             if sale_rev != 0:
#                 sale_profit_pct = (sale_rev / sale_cost - 1) * 100
#                 print(sale_profit_pct)
#                 if sale_profit_pct > 0:
#                     correct += 1
#                 else:
#                     wrong += 1
#         amount = 0
#         sale_cost = 0
# print(correct/(correct+wrong))

Bought 1 at an average of 46952.0
Sold 1 at 47588.42
1.3554694155733404
Bought 1 at an average of 46989.28
Bought 2 at an average of 46921.095
Sold 2 at 46555.42
-0.7793402945945771
Bought 1 at an average of 46395.9
Sold 1 at 47152.72
1.6312217243334093
Bought 1 at an average of 46412.41
Sold 1 at 48685.19
4.896923042781021
Bought 1 at an average of 48614.97
Sold 1 at 50994.75
4.8951588368767895
Bought 1 at an average of 50049.34
Sold 1 at 50791.11
1.4820774859368946
Bought 1 at an average of 47987.57
Sold 1 at 46824.23
-2.4242527804596015
Bought 1 at an average of 46896.5
Bought 2 at an average of 47120.665
Sold 2 at 47280.01
0.33816373347024253
Bought 1 at an average of 47192.55
Sold 1 at 46910.3
-0.5980816887411256
Bought 1 at an average of 47126.02
Sold 1 at 45922.0
-2.5548943025530235
Bought 1 at an average of 46367.53
Bought 2 at an average of 46395.04
Sold 2 at 46414.47
0.04187947677165571
Bought 1 at an average of 42878.34
Sold 1 at 41803.11
-2.507629726337346
Bought 1 at an av

In [10]:
count = 0
correct = 0
bought = 0
sale_cost = 0
total_profit = 0
balance = 100
buy_money = balance / 5
while count < 100:
    tracking_data = get_ta(get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '500'))
    print("Current price is {} and macd 1min is {}, prev macd 1min is {}".format(tracking_data.iloc[-1, 5], tracking_data.iloc[-2,10], tracking_data.iloc[-2,11]))

    buy_cond_1 = (tracking_data.iloc[-2,10] > 0) & (tracking_data.iloc[-2,11] < 0) 
    buy_cond_2 = (tracking_data.iloc[-2,8] < 0) & (tracking_data.iloc[-2,9] < 0)

    if buy_cond_1 & buy_cond_2:
        price = tracking_data.iloc[-1, 5]
        if balance >= buy_money:
            bought += buy_money/price
            sale_cost += buy_money
            balance -= buy_money
        else:
            bought += balance/price
            sale_cost += balance
            balance = 0
        print("Bought {} at an average of {}".format(bought, sale_cost/bought))
        print("Remaining balance :{}".format(balance))
    
    sell_cond_1 = (tracking_data.iloc[-2,10] < 0) & (tracking_data.iloc[-2,11] > 0)
    sell_cond_2 = (tracking_data.iloc[-2,8] > 0) & (tracking_data.iloc[-2,9] > 0)

    if sell_cond_1 & sell_cond_2:
        if bought != 0:
            count += 1
            print("Sold {} for a total of {}".format(bought, bought*tracking_data.iloc[-1, 5]))
            balance += tracking_data.iloc[-1, 5] * bought
            buy_money = balance / 5
            if tracking_data.iloc[-1, 5] * bought > sale_cost:
                correct += 1
                print("{}/{}".format(correct, count))
            print(balance)
            bought = 0
            sale_cost = 0
    time.sleep(60)
        
    
print(total_profit)
print(correct/count)

Current price is 16860.36 and macd 1min is 0.021911547617632365, prev macd 1min is -0.7422655686248696
Bought 0.0011862142919842754 at an average of 16860.36
Remaining balance :80.0
